# Load Packages

In [ ]:
# Add any needed packages
import plotly.express as px
from google.cloud import bigquery
import networkx as nx
from collections import defaultdict
from pyvis.network import Network
from tqdm import tqdm
from scipy.interpolate import interp1d
import numpy as np

# Setup Connection

In [ ]:
# Fill in identifiers

CURATION_PROJECT_ID = ""
EHR_OPS_DATASET_ID = ""
RDR_PROJECT_ID = ""
RDR_DATASET_ID = ""
COMBINED_DATASET_ID = ""

# Add any other needed identifiers

In [ ]:
client = bigquery.Client(project=RDR_PROJECT_ID)

## Smoked 100 Cigs entire life

Description: Pulls the first 10 rows in the measurement table

In [ ]:
# Define Query 1
q1 = f"""
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
value_as_concept_id,
c1.concept_name,
count(distinct person_id) as pt_cnt,
count(distinct person_id)/ (select count(distinct person_id) from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation`) as pt_percent
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 40766306
group by 1,2,3,4
"""

In [ ]:
results1 = client.query(q1).to_dataframe()
results1

## If Yes, do you now smoke cigarettes every day, some days, or not at all?

Description: Pulls the first 10 rows in the procedure_occurrence table

In [ ]:
# Define Query 2

q2 = f"""
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
value_as_concept_id,
c1.concept_name,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 40766307
group by 1,2,3,4
"""

In [ ]:
results2 = client.query(q2).to_dataframe()
results2['pt_percent'] = results2['pt_cnt']/results1[results1['concept_name'] == 'Yes']['pt_cnt'].values[0]
results2

In [ ]:
q_smoker = f"""
select distinct person_id, 1 as current_smoker
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 40766307
and value_as_concept_id in (45881677, 45883381)
union distinct
select distinct person_id, 0 as current_smoker
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 40766307
and value_as_concept_id in (45883172)
"""

In [ ]:
current_smoking = client.query(q_smoker).to_dataframe()

In [ ]:
current_smoking.groupby('current_smoker').count()

In [ ]:
results2['pt_percent'].sum()

## On average, how many cigarettes do you smoke per day now? (There are 20 cigarettes in a pack.)

In [ ]:
q3 = f"""
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
c1.concept_name,
NULL as value_as_number,
value_as_concept_id,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 40766929
--and value_as_concept_id is not null
group by 1,2,3,4,5
union all
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
'' as concept_name,
value_as_number,
NULL as value_as_concept_id,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
left join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 40766929
--and value_as_number != 0
--and value_as_number is not null
group by 1,2,3,4,5
order by pt_cnt desc
"""

In [ ]:
results3 = client.query(q3).to_dataframe()
results3 = results3[results3['value_as_concept_id'].notnull() | results3['value_as_number'].notnull()]
results3['pt_percent'] = results3['pt_cnt']/results1[(results1['concept_name'] == 'Yes')]['pt_cnt'].sum()

In [ ]:
results3['pt_percent'].sum()

In [ ]:
results3['pt_cnt'].sum()

In [ ]:
at_least_1_per_day = results3[results3['value_as_number'] >= 1]['pt_cnt'].sum()
print(f'people who smoked at least 1 cigs per day: {at_least_1_per_day} \npercentage of people who smoked at least 100 cigs: {at_least_1_per_day/results3["pt_cnt"].sum()}')

In [ ]:
no_per_day = results3[results3['value_as_number'] == 0]['pt_cnt'].sum()
print(f'people who smoked 0 cig per day {no_per_day} \npercentage of people who smoked at least 100 cigs {no_per_day/results3["pt_cnt"].sum()}')

In [ ]:
skip_unknown = results3[results3['value_as_concept_id'].notnull()]['pt_cnt'].sum()

In [ ]:
print(f'people who smoked 0 cig per day {skip_unknown} \npercentage of people who smoked at least 100 cigs {skip_unknown/results3["pt_cnt"].sum()}')

In [ ]:
fig1 = px.bar(results3[results3['value_as_number'].notnull()][['value_as_number','pt_cnt']], x='value_as_number', y='pt_cnt',
              labels={'value_as_number':'Number of cigarettes smoked per day',
                      'pt_cnt':'Number of participants'}, log_y=False)

In [ ]:
fig1.show()

### How many years have you or did you smoke cigarettes?

In [ ]:
q4 = f"""
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
c1.concept_name,
NULL as value_as_number,
value_as_concept_id,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 40770349
--and value_as_concept_id is not null
group by 1,2,3,4,5
union all
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
'' as concept_name,
value_as_number,
NULL as value_as_concept_id,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
left join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 40770349
group by 1,2,3,4,5
order by pt_cnt desc
"""

In [ ]:
results4 = client.query(q4).to_dataframe()


In [ ]:
results4

In [ ]:
fig = px.bar(results4[results4['value_as_number'].notnull()][['value_as_number','pt_cnt']], x='value_as_number', y='pt_cnt',
             labels={'value_as_number':'Number of years smoked',
                     'pt_cnt':'Number of participants'})

In [ ]:
fig.show()

### On average, over the entire time that you smoked, how many cigarettes did you smoke each day? (There are 20 cigarettes in a pack.)

In [ ]:
q5 = f"""
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
c1.concept_name,
NULL as value_as_number,
value_as_concept_id,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 40766930
--and value_as_concept_id is not null
group by 1,2,3,4,5
union all
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
'' as concept_name,
value_as_number,
NULL as value_as_concept_id,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
left join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 40766930
group by 1,2,3,4,5
order by pt_cnt desc
"""

In [ ]:
results5 = client.query(q5).to_dataframe()

In [ ]:
results5

In [ ]:
fig = px.bar(results5[results5['value_as_number'].notnull()][['value_as_number','pt_cnt']], x='value_as_number', y='pt_cnt')

In [ ]:
fig.show()

### In the past, have you ever made a serious attempt to quit smoking? That is, have you stopped smoking for at least one day or longer because you were trying to quit?

In [ ]:
q6 = f"""
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
value_as_concept_id,
c1.concept_name,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 40766643
group by 1,2,3,4
"""

In [ ]:
results6 = client.query(q6).to_dataframe()

In [ ]:
results6

In [ ]:
results6['pt_percent'] = results6['pt_cnt']/results1[results1['concept_name'] == 'Yes']['pt_cnt'].values[0]

In [ ]:
results6

### If you have completely stopped smoking cigarettes, how old were you when you stopped?

In [ ]:
q7 = f"""
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
c1.concept_name,
NULL as value_as_number,
value_as_concept_id,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 1585870
--and value_as_concept_id is not null
group by 1,2,3,4,5
union all
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
'' as concept_name,
value_as_number,
NULL as value_as_concept_id,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
left join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 1585870
group by 1,2,3,4,5
order by pt_cnt desc
"""

In [ ]:
results7 = client.query(q7).to_dataframe()

In [ ]:
results7

In [ ]:
fig = px.bar(results7[results7['value_as_number'].notnull()][['value_as_number','pt_cnt']], x='value_as_number', y='pt_cnt',
             labels={'value_as_number':'Age at Quitting Smoking', 'pt_cnt':'Participant Count'})

In [ ]:
fig.show()

### Electronic Smoking

In [ ]:
q8 = f"""
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
value_as_concept_id,
c1.concept_name,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 1586166
group by 1,2,3,4
"""

In [ ]:
results8 = client.query(q8).to_dataframe()

In [ ]:
results8['pt_percent'] = results8['pt_cnt']/results8['pt_cnt'].sum()

In [ ]:
results8

### Have you ever smoked tobacco in a hookah, even one or two puffs?

In [ ]:
q9 = f"""
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
value_as_concept_id,
c1.concept_name,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 1586182
group by 1,2,3,4
"""

In [ ]:
results9 = client.query(q9).to_dataframe()

In [ ]:
results9['pt_percent'] = results9['pt_cnt']/results9['pt_cnt'].sum()

In [ ]:
results9

### Have you ever smoked a traditional cigar, cigarillo, or filtered cigar, even one or two puffs?

In [ ]:
q10 = f"""
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
value_as_concept_id,
c1.concept_name,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 1586174
group by 1,2,3,4
"""

In [ ]:
results10 = client.query(q10).to_dataframe()

In [ ]:
results10['pt_percent'] = results10['pt_cnt']/results10['pt_cnt'].sum()

In [ ]:
results10

### Occupational lung disorder

In [ ]:
q11 = f"""
select distinct
condition_concept_id,
c1.concept_name as condition_concept_name,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.condition_occurrence` c
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on c.condition_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept_ancestor` c2
on c.condition_concept_id = c2.descendant_concept_id
where c2.ancestor_concept_id = 4312754
group by 1,2
"""

In [ ]:
results11 = client.query(q11).to_dataframe()

In [ ]:
results11['pt_percent'] = results11['pt_cnt']/results11['pt_cnt'].sum()

In [ ]:
results11

### Obstructive Lung Disease Hx (Asthma, COPD, bronchitis)
#### Are you still seeing a doctor or health care provider for chronic lung disease (COPD, emphysema, or bronchitis)?

In [ ]:
q12 = f"""
select distinct
o.observation_concept_id,
c1.concept_name as observation_concept_name,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.observation_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept_ancestor` c2
on o.observation_concept_id = c2.descendant_concept_id
where c2.ancestor_concept_id = 1384608
group by 1,2
"""

In [ ]:
results12 = client.query(q12).to_dataframe()

In [ ]:
results12['pt_percent'] = results12['pt_cnt']/results12['pt_cnt'].sum()

In [ ]:
results12

#### About how old were you when you were first told you had chronic lung disease (COPD, emphysema, or bronchitis)?

In [ ]:
q13 = f"""
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
c1.concept_name,
value_as_concept_id,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 1384393
--and value_as_concept_id is not null
group by 1,2,3,4
"""

In [ ]:
results13 = client.query(q13).to_dataframe()

In [ ]:
results13['pt_percent'] = results13['pt_cnt']/results13['pt_cnt'].sum()

In [ ]:
results13

In [ ]:
fig = px.pie(results13, values='pt_cnt', names='concept_name', title='Age of first diagnosis of COPD, emphysema, or bronchitis')

In [ ]:
fig.show()

#### Are you still seeing a doctor or health care provider for asthma?

In [ ]:
q14 = f"""
select distinct
o.observation_concept_id,
c1.concept_name as observation_concept_name,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.observation_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept_ancestor` c2
on o.observation_concept_id = c2.descendant_concept_id
where c2.ancestor_concept_id = 43530388
group by 1,2
"""

In [ ]:
results14 = client.query(q14).to_dataframe()

In [ ]:
results14['pt_percent'] = results14['pt_cnt']/results14['pt_cnt'].sum()

In [ ]:
results14

#### About how old were you when you were first told you had asthma?

In [ ]:
q15 = f"""
select distinct
observation_concept_id,
c2.concept_name as observation_concept_name,
c1.concept_name,
value_as_concept_id,
count(distinct person_id) as pt_cnt
from `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` o
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c1
on o.value_as_concept_id = c1.concept_id
join `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c2
on o.observation_concept_id = c2.concept_id
where observation_concept_id = 43530546
--and value_as_concept_id is not null
group by 1,2,3,4
"""

In [ ]:
results15 = client.query(q15).to_dataframe()

In [ ]:
results15['pt_percent'] = results15['pt_cnt']/results15['pt_cnt'].sum()

In [ ]:
results15

In [ ]:
fig = px.pie(results15, values='pt_cnt', names='concept_name', title='Age at first asthma diagnosis')

In [ ]:
fig.show()

### Network Graph

In [ ]:
copd_social_family_hx_concept_ids = [
    1585856, # Lifestyle - Smoking
    1384561 # Personal Medical History: Respiratory
]

In [ ]:
q_concept = f"""

WITH RECURSIVE
  base_concepts AS (
    SELECT
      *
    FROM `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` c
    WHERE c.concept_id IN (1585856, 1384561)
  ),
  concept_children AS (
      SELECT
          concept_id child_concept_id, NULL parent_concept_id
      FROM base_concepts
      UNION ALL
      SELECT cr.concept_id_1 child_concept_id, cr.concept_id_2 parent_concept_id
      FROM concept_children par
      JOIN `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept_relationship` cr
          ON cr.concept_id_2 = par.child_concept_id
              AND cr.relationship_id = 'Is a'
  )
  SELECT DISTINCT
    child_concept_id, child_concept.concept_name child_concept_name, child_concept.domain_id child_concept_domain,
    parent_concept_id, parent_concept.concept_name parent_concept_name, parent_concept.domain_id parent_concept_domain
  FROM concept_children cc
  JOIN `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` child_concept
    ON child_concept.concept_id = cc.child_concept_id
  JOIN `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` parent_concept
    ON parent_concept.concept_id = cc.parent_concept_id
"""

results = client.query(q_concept).to_dataframe()
results

In [ ]:
# Define Query 1
q_nx = f"""
    SELECT
        t.observation_id event_id, par_c.domain_id domain_id, t.observation_date event_date,
        par_c.concept_id ancestor_concept_id, par_c.concept_name ancestor_concept_name,
        chd_c.concept_id child_concept_id, chd_c.concept_name child_concept_name
    FROM `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.observation` t
    JOIN `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` chd_c
      ON chd_c.concept_id = t.observation_concept_id
    JOIN `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept_ancestor` ca
      ON ca.descendant_concept_id = chd_c.concept_id
    JOIN `{CURATION_PROJECT_ID}.{COMBINED_DATASET_ID}.concept` par_c
      ON par_c.concept_id = ca.ancestor_concept_id
        AND par_c.concept_id IN ({", ".join(map(str, copd_social_family_hx_concept_ids))})
        AND par_c.domain_id = 'Observation'

"""

In [ ]:
copd_concept_events = client.query(q_nx).to_dataframe()
copd_concept_events

In [ ]:
dict(copd_concept_events.child_concept_id.value_counts())

### Build Network Graph

In [ ]:
def build_nx_graph(rel_df, concept_counts={}):
    rel_df['child_concept_id'] = rel_df['child_concept_id'].astype(int)
    rel_df['parent_concept_id'] = rel_df['parent_concept_id'].astype(int)

    colors = defaultdict(lambda: 'gray')
    colors.update({'Condition': 'blue', 'Procedure': 'red'})
    G = nx.DiGraph()

    interp_concept_counts = {}
    if concept_counts:
        f = np.interp(list(concept_counts.values()),
            [0,
             np.median(list(concept_counts.values())),
             max(concept_counts.values())], [1, 10, 20])

        interp_concept_counts = dict(zip(list(concept_counts.keys()), f))

    for i, rel in tqdm(rel_df.iterrows()):
        child_node_size = 1
        child_count = 0
        parent_node_size = 1
        parent_count=0

        if interp_concept_counts:
            if rel['child_concept_id'] in interp_concept_counts:
                child_node_size = interp_concept_counts[
                    rel['child_concept_id']]

                child_count = concept_counts[rel['child_concept_id']]

            if rel['parent_concept_id'] in interp_concept_counts:
                parent_node_size = interp_concept_counts[
                    rel['parent_concept_id']]

                parent_count = concept_counts[rel['parent_concept_id']]

        G.add_node(str(rel['child_concept_id']),
                   concept_id=rel['child_concept_id'],
                   concept_name=rel['child_concept_name'],
                   title=f"{rel['child_concept_name']}\nCount: {child_count}",
                   domain_id=rel['child_concept_domain'],
                   color=colors[rel['child_concept_domain']],
                   size=child_node_size,
                  )

        G.add_node(str(rel['parent_concept_id']),
                   concept_id=rel['parent_concept_id'],
                   concept_name=rel['parent_concept_name'],
                   title=f"{rel['parent_concept_name']}\nCount: {parent_count}",
                   domain_id=rel['parent_concept_domain'],
                   color=colors[rel['parent_concept_domain']],
                   size=parent_node_size)

        G.add_edge(rel['parent_concept_id'], rel['child_concept_id'], id=i)

    return G

In [ ]:
nx_graph = build_nx_graph(results, dict(copd_concept_events.child_concept_id.value_counts()))

In [ ]:
nt = Network('1000px', '1000px', notebook=True, directed=True, cdn_resources='in_line')
nt.from_nx(nx_graph)
nt.repulsion()
nt.show('nx.html')